<a href="https://colab.research.google.com/github/davis689/binder/blob/master/heat_of_isochoric_temperature_change.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

First we setup some system stuff to handle symbolic manipulations.

In [1]:
pip install sympy # if sympy is already installed, this step can be commented out.

In [2]:
import sympy as sp
sp.init_printing()

## The problem
An ideal gas is warms from 298 K and 1bar to 308 K at constant volume. Calculate the heat involved in the change. 

First we will define some variables and constants. These can be defined all together but it will sometimes be convenient to specify these as fully as possible so that the system knows how to treat them and here we'll keep the constants separate from the variables.

In [3]:
p,V,T=sp.symbols("p,V,T",real=True,positive=True) 
R,Cv,Cp=sp.symbols("R,Cv,Cp",real=True,positive=True,constant=True) 
ps,Ts,Vs=[],[],[] # define some variables to contain the list of beginning, intermediate, and ending pressures (p's), temperatures (T's), and volumes (V's)
Rs=[0.08314,8.314] # Define two R values. R[0] will be in Lbar/molK and R[1] will be in J/molK. We need to remember to use the appropriate one.
Cs=[12.471,20.785] #Cv and Cp


Next we'll define the temperatures and pressures at the beginning and end of the two steps involved in our process.

In [4]:
Ts=[298,308] # set up the temperatures for the beginning, intermediate, and ending points.
Vs=Rs[0]*Ts[0]/1 # Calculate volume from initial T and p so calculate from ideal gas law using R=0.08314 and T=298

Now we can setup the equation of state. This is not strictly necessary since we could always manually manipulate the equation (especially the ideal gas equation) and input the form we want into the expression we need to calculate, but this will give us flexibility to change the equation without changing any of the following code if we so desire.

In [5]:
ig=sp.Eq(p*V,R*T) #ideal gas equation. 

Now we'll use the equation to solve for the pressures at the beginning and end using the temperatures and volume we declared above. We first solve the equation, ig, for p and then substitute into it the values for V, T, and R.

In [6]:
ps=[sp.solve(ig,p)[0].subs(V,Vs).subs(T,Ts[i]).subs(R,Rs[0]) for i in range(len(Ts))] # range(len(ps)) will make a list starting from 0 with as many members as are in ps
                                                                                      # sp.solve(ig,p)[0] solves ig for p. The [0] extracts the, in this case, only solution from the solutions list.
                                                                                      # the subs arguments substitute for V, T, and R the values we defined earlier for each step in the process.


Now to check that the solving and substitution (and unit manipulation) went as expected, we print the results. 

In [7]:
print([ps[i].round(5) for i in range(len(Ts))]) #print the pressures to make sure they make sense
#ps[i].round(5) rounds to 5 decimal places the i-th result in ps
# for i in range(len(Ts)) iterates over i=0 up to but not including whatever the length the the Ts vector is. In this case it is 2 so i=0 and 1.

[1.0000, 1.03356]


## 1 - in terms of dT and dp
Let's calculate the heat using $$dq=C_p dT + l_p dp$$ The pressure and temperature are both changing so we'll need to calculate both terms. $$l_p=-T\left(\dfrac{\partial V}{\partial T}\right)$$

In [32]:
lp=-T*sp.diff(sp.solve(ig,V)[0],T) # find lp.
display(lp) # before integrating with respect to p, we need to subsitute T as a function of p since both ar changing.
qa=sp.integrate(lp.subs(T,sp.solve(ig,T)[0]),(p,ps[0],ps[1]))*100 # calculate the heat by integrating wrt pressure after substituting T in terms of p. 
qb=sp.integrate(Cs[1],(T,Ts[0],Ts[1])) # integrate Cp wrt T
q1=qa.subs(R,Rs[0]).subs(V,Vs).evalf()+qb # substitute R and V, evaluate, and save as q1.

In [33]:
print("The heat transferred due to pressure change is ", qa.subs(R,Rs[0]).subs(V,Vs).evalf().round(2)," J/mol")
print("The heat transferred due to pressure change is ", qb.round(2)," J/mol")
print("The total heat in this isochoric path is ",q1.round(2)," J/mol")


The heat transferred due to pressure change is  -83.14  J/mol
The heat transferred due to pressure change is  207.85  J/mol
The total heat in this isochoric path is  124.71  J/mol


## 2 - In terms of dT and dV
Let's calculate the same change using a different heat equation. $$dq=C_VdT+l_vdV$$

In [25]:
lv=T*sp.diff(sp.solve(ig,p)[0],T)
qa=sp.integrate(Cs[0],(T,Ts[0],Ts[1])) # integrate wrt T
qb=sp.integrate(lv,(V,Vs,Vs)) #Integrate wrt V (a bit boring since V doesn't change)
q2=qa.subs(R,Rs[1]).subs(T,Ts[0]).evalf()+qb


In [26]:
print("The heat transferred due to temperature change is ", qa.subs(R,Rs[1]).subs(T,Ts[0]).evalf().round(2)," J/mol")
print("The heat transferred due to volume change is ", qb.round(2)," J/mol")
print("The total heat in this isochoric path is ",q2.round(2)," J/mol")
print("The total heat using this heat equation, ",q2.round(2),", should turn out to be identical to the heat using the other heat equation,",q1.round(2),". If not, we've done something wrong.")

The heat transferred due to temperature change is  124.71  J/mol
The heat transferred due to volume change is  0  J/mol
The total heat in this isochoric path is  124.71  J/mol
The total heat using this heat equation,  124.71 , should turn out to be identical to the heat using the other heat equation, 124.71 . If not, we've done something wrong.


## 3 - In terms of dp and dV
One more time with the last heat equation...  $ dq=\gamma_p dp + \gamma_V dV$. 


In [27]:
gamp=Cv/sp.diff(sp.solve(ig,p)[0],T) # define gamma p and gamma v using techniques we can learn later
gamv=Cp/sp.diff(sp.solve(ig,V)[0],T) 
qa=sp.integrate(gamp,(p,ps[0],ps[1])) #integrate wrt p
qb=sp.integrate(gamv,(V,Vs,Vs)) #integrate wrt V
q3=qa.subs(R,Rs[0]).subs(Cv,Cs[0]).subs(V,Vs)+qb


In [28]:
print("The heat transferred due to pressure change is ", qa.subs(R,Rs[0]).subs(Cv,Cs[0]).subs(V,Vs).round(2)," J/mol")
print("The heat transferred due to volume change is ", qb.round(2)," J/mol")
print("The total heat in this isochoric path is ",q3.round(2)," J/mol")
print("The total heat using this heat equation, ",q3.round(2),", should turn out to be identical to the heat using the other heat equation,",q1.round(2),". If not, we've done something wrong.")

The heat transferred due to pressure change is  124.71  J/mol
The heat transferred due to volume change is  0  J/mol
The total heat in this isochoric path is  124.71  J/mol
The total heat using this heat equation,  124.71 , should turn out to be identical to the heat using the other heat equation, 124.71 . If not, we've done something wrong.
